In [18]:
from datetime import datetime

import pandas as pd
from utils import get_location_details

In [19]:
def process_data(data):
    if "m2" in data:
        return int(data.replace("m2", ""))
    elif "Ha" in data:
        return int(data.replace("Ha", "")) * 10000

    else:
        return data

In [20]:
def convert_to_int(value):
    # Try to convert the value to an integer
    try:
        return int(value)
    except ValueError:
        # If a ValueError is raised, remove non-numeric characters and try again
        return int("".join(filter(str.isdigit, str(value))) or 0)

In [21]:
def date_correction(date):
    return date - 2 * (date - datetime.now())

In [22]:
# Read the CSV file
df = pd.read_csv("../terrain_coinafrique.csv", skiprows=1)

In [23]:
df

,Parcelle 621m²- Agla,"Détails du produitNous mettons en vente à agla en face brigade des moeurs,\n\nen 3 ème position des pavés, \nclôturée avec portail, \n\ncontenant quelques constructions conservables, \n\nsuperficie : 621 m²\n\ndocuments : attestation de recensement + levé topographique + convention \n\nprix: 45.000.000 f à revoir légèrement.\n\npropriétaire vivant",40 000 000 CFA,622 m2,2025-02-19 19:14:35.150679
0,Parcelle - Glo,Détails du produitParcelle size à glo dans la ...,25 000 000 CFA,3600 m2,2024-02-20 19:14:37.286700
1,Terrains 350m² - Abomey-calavi,Détails du produitBonjour. notre cabinet met e...,12 000 000 CFA,350,2024-02-20 19:14:38.230361
2,Parcelle 325 m2 - Akpakpa Avotrou,Détails du produitNous mettons en vente a akpa...,25 000 000 CFA,325 m2,2024-02-20 19:14:39.437170
3,Parcelles 631m²- Sagla,Détails du produitNous mettons en vente pour u...,45 000 000 CFA,684 m2,2024-02-21 19:14:40.405320
4,Parcelles - Cotonou,Détails du produitVraie opportunité à cotonou ...,16 000 000 CFA,1 m2,2024-02-21 19:14:41.551877
...,...,...,...,...,...
15244,Vente terrains 350 m2 - Cotonou Aibatin,Détails du produit505p14 parcelle à vend...,28 000 000 CFA,350,2027-02-20 16:50:28.122326
15245,Vente terrains 10000 - Allada,Détails du produit 1 hectare à Allada Avakpa. ...,1 800 000 CFA,10000,2027-02-20 16:50:29.367857
15246,Vente Terrains 10000 - Kétou,Détails du produit505p09 58 hectares av...,900 000 CFA,10000,2027-02-20 16:50:30.615252
15247,Vente terrains 10000 m2 - Kétou,Détails du produitDisponible un domaine de 10m...,1 000 000 CFA,10000,2027-02-20 16:50:31.862535


In [24]:
# Remove duplicate rows
df = df.drop_duplicates()
# Set the column names
df.columns = ["title", "description", "price", "area", "date"]

In [25]:
# Delete  "m2" on column "area" and
df["area"] = df["area"].apply(process_data)
# Convert the "area" column to integers
df["area"] = df["area"].apply(convert_to_int)
df["price"] = df["price"].apply(convert_to_int)

In [26]:
# Delete the column with price < 10000 or area < 100
df = df[df["price"] > 10000]
df = df[df["area"] > 100]

In [27]:
df["date"] = pd.to_datetime(df["date"])
df["date"] = df["date"].apply(date_correction)

In [28]:
# reduce date to years and months
df["date"] = df["date"].dt.strftime("%Y-%m")
# Remove duplicate rows
df = df.drop_duplicates()

In [29]:
# Transform title column on two columns
df[["title", "city"]] = df["title"].str.split("-", n=1, expand=True)
# Delete the column with city null
df = df[df["city"].notna()]

In [30]:
# suppress spaces at the beginning of the city column
df["city"] = df["city"].str.strip()

In [31]:
# transform the city column to lowercase
df["city"] = df["city"].str.lower()

In [42]:
# get the location details
def location(city):
    answer = get_location_details(city)
    if answer:
        clean_answer = answer.split(", Bénin")
        return clean_answer[0] + ", Bénin"
    else:
        city_clean = city.split(" ")
        answer = get_location_details(city_clean[0])
        if answer:
            clean_answer = answer.split(", Bénin")
            return clean_answer[0] + ", Bénin"
        else:
            try:
                print(city_clean[0])
                return get_location_details(city_clean[1])
            except:
                return "None"

In [43]:
# Get the location details and save on the 4 new column
df["location"] = df["city"].apply(lambda x: location(x))

sagla
cococotomey
hevier
sikèkodji
calavi-ouèdo
Terrain de Football SONICOG, Rue 40, Gbedji, Porto-Novo, Porto Novo, Ouémé, Bénin
sikècodji
sikecodji
sikecodji
sikecodji
kpota-ouèdo
zoundia
adjavié
sikecoddji
mariagleta
fidjirossé
djiegan
Kraké-Daho, Sèmè-Kpodji, Ouémé, Bénin
Adounko Daho, Abomey-Calavi, Atlantique, Bénin
Houakpè Daho, Ouidah, Atlantique, Bénin
Togbin Daho, Abomey-Calavi, Atlantique, Bénin
Djako Daho, Abomey-Calavi, Atlantique, Bénin
Assogbénou-Daho, Kpomassè, Atlantique, Bénin
Dèwémè-Daho, Houédomey, Dangbo, Ouémé, Bénin
Tokpa-Daho, Adja-Tokpa, Kpomassè, Atlantique, Bénin
Agonvodji-Daho, Nazoumè, Kpomassè, Atlantique, Bénin
Vèkky-Daho, Vèkky, Sô-Ava, Atlantique, Bénin
cavali
abritant
ex
Bénin
acronville
jonckey
dodja
assorclé
ounmakô
houehiho
akpapka
cocodji
attrokpocodji
Carrefour zico, Cocotomey, Abomey-Calavi, Atlantique, Bénin
von zico, Cocotomey, Abomey-Calavi, Atlantique, Bénin
attrokpocodji
fidjrossè-togbin
acadjamey
cavali
cococotomey
kpatalokoli
atropocodji
r

In [44]:

df

,title,description,price,area,date,city,location
0,Parcelle,Détails du produitParcelle size à glo dans la ...,25000000,3600,2024-02,glo,"GLO, Rue 2147, Hlazounto, 12ᵉ Arrondissement, ..."
1,Terrains 350m²,Détails du produitBonjour. notre cabinet met e...,12000000,350,2024-02,abomey-calavi,"Abomey-Calavi, Atlantique, Bénin"
2,Parcelle 325 m2,Détails du produitNous mettons en vente a akpa...,25000000,325,2024-02,akpakpa avotrou,"Imoro, Ikpinlè, Adja-Ouèrè, Plateau, Bénin"
3,Parcelles 631m²,Détails du produitNous mettons en vente pour u...,45000000,684,2024-02,sagla,None
5,Parcelles,Détails du produitIl est mis en vente à hêvié ...,3200000,500,2024-02,hevié,"Hêvié, Abomey-Calavi, Atlantique, Bénin"
...,...,...,...,...,...,...,...
8881,Vente Terrain 450m²,Détails du produitVente d’un ensemble de 10 pa...,2699000,450,2021-02,hevié centre,"Centre de santé de Hêvié, RN 30, Somè, Ouèdo, ..."
8882,Vente parcelle 500m²,Détails du produitVente de parcelles de 500m² ...,890000,500,2021-02,zinvié zoumey,"RN 31, Zinvié, Abomey-Calavi, Atlantique, Bénin"
8883,Vente parcelles 350 m²,Détails du produitVente d’un ensemble de 22 pa...,399000,350,2021-02,kpomassè,"Kpomassè, Atlantique, Bénin"
8887,Vente Terrain 501 m²,Détails du produitParcelle de superficie: 501m...,50000000,501,2021-02,cotonou,"Avenue Codje Tovalou Quénum, 7ème Arrondisseme..."


In [45]:
# Write the DataFrame back to the CSV file
df.to_csv("../terrain_coinafrique_clean.csv", index=False)